In [1]:
#O problema pode ser modelado da seguinte maneira:
#buscamos maximizar a função objetivo quantidade de carga f(x,y)=american_cargo*x+british_cargo*y,em que x é o número de aviões
#americanos e y é o número de aviões britânicos, sujeitas às restrições:
#1) 0 <= r1(x,y) <= max_planes, onde r1(x,y)=x+y;
#2) 0 <= r2(x,y) <= max_pers, onde r2(x,y)=american_pers*x+british_pers*y;
#3) 0 <= r3(x,y) <= budget, onde r3(x,y)=american_cost*x+british_cost*y.

#Este é um problema de programação linear. A solução pelo método Simplex não é possível, pois este fica iterando em um loop
#indefinidamente, devido à aparição de uma constante positiva na linha da função objetivo na tabela do método.

#Portanto resolveremos pelo seguinte método: o teorema fundamental da programação linear garante que os pontos de máximo
#da função somente podem ser atingidos na borda da região da restrição.
#A região restrição é dada pela intersecção, no quadrante superior direito, das áreas abaixo (e incluindo) das retas
#r1: y = max_planes - x
#r2: y = max_pers/british_pers - (american_pers/british_pers)*x
#r3: y = budget/british_cost -(american_cost/british_cost)*x

#Como nenhuma destas retas coincide com as retas das curvas de níveis - que são da forma, para um valor k,
#y = k/british_cargo -(american_cargo/british_cargo)*x -,
#segue que o ponto de máximo deve estar em um dos vértices da região.

#Para achá-los, basta tomar os 10 pontos de todas as intersecções possíveis entre r1, r2, r3, Ox, Oy, em que Ox, Oy são
#o eixo das abscissas e o eixo das coordenadas, respectivamente.

#Em seguida, consideramos dentre estes os pontos possíveis (feasible points) que estão na região restrição restrição; basta,
#então, calcular o valor da função objetivo nestes pontos e determinaremos o ponto de máximo da função.

#Este método é computacionalmente viável, pois este é um problema em dimensão n=2 com apenas 3 restrições.


In [2]:
import numpy as np

In [3]:
#Função objetivo
def cargo(X, american_cargo, british_cargo):
    return american_cargo*X[0]+british_cargo*X[1]

In [4]:
def plane_model(budget, max_planes=44, max_pers=512,
                  american_cargo=30000.,american_pers=16, american_cost=9000.,
                  british_cargo=20000., british_pers=32, british_cost=5000.):
    #Acharemos as intersecções das retas da seguinte maneira: se r e s são as retas definidas por
    #a1*x + a2*y = b1
    #a3*x + a4*y = b2,
    #resolvemos o sistema linear AX = B, em que X é o vetor coluna (x,y), A é a matriz
    #(a1 a2)
    #(a3 a4)
    #e B é o vetor coluna (b1,b2).
    
    #O código a seguir prepara uma interação compacta para resolver os sistemas lineares considerados.
    r1_A = np.array([1,1])
    r1_B = max_planes  
    r2_A = np.array([american_pers, british_pers])
    r2_B = max_pers
    r3_A = np.array([american_cost, british_cost])
    r3_B = budget

    abs_A = np.array([0,1])
    abs_B = 0
    ords_A = np.array([1,0])
    ords_B = 0

    vec_A = [r1_A, r2_A, r3_A, abs_A, ords_A]
    pt_B = [r1_B, r2_B, r3_B, abs_B, ords_B]

    solutions = []
    for i in range(len(vec_A)):
        for j in range(i+1, len(pt_B)):
            A = np.vstack((vec_A[i], vec_A[j]))
            B = np.vstack((pt_B[i], pt_B[j]))
            solutions.append(np.linalg.solve(A,B))        
    feasible_solutions = [X for X in solutions if X[0]>=0 and X[1]>=0 
                      and np.dot(r1_A, X)<=r1_B and np.dot(r2_A,X)<=r2_B and np.dot(r3_A,X)<=r3_B]
    
    return np.reshape(feasible_solutions[np.argmax([cargo(X, american_cargo, british_cargo) 
                                                    for X in feasible_solutions])], 2).astype('int')
#A última linha seria mais clara se fosse dividida em 3 ou 4 linhas, mas assim temos um pequeno ganho computacional e 
#de uso de memória.
#Retornamos o vetor X transformado em um vetor linha e atribuimos o tipo 'int' ao vetor (x,y),
#o que retorna o maior inteiro menor ou igual ao número considerado, ou seja, arredonda sempre para baixo.

#Observação: Essa pode nem sempre ser a melhor escolha de arredondamento. Pode-se facilmente escrever um pequeno algoritmo 
#que analisa os inteiros próximos a (x,y) e retorna, dentre esses, o par de inteiros que está na região restrição e é 
#tal que seu valor da função objetivo seja o maior.
#Aqui isso não é feito apenas para maior clareza no código.

In [5]:
#Apenas uma função que printa as informações do modelo. Poderíamos fazer uma -ou diversas- função que retorna as informações
#do modelo, para utilizar informações mais específicas dentro do sistema.
def plane_info(american_planes, british_planes, budget, max_planes=44, max_pers=512,
                  american_cargo=30000.,american_pers=16, american_cost=9000.,
                  british_cargo=20000., british_pers=32, british_cost=5000.):
    print('Informações')
    print('Cargo:', cargo([american_planes, british_planes], american_cargo, british_cargo))
    print('Budget limite:', budget)
    print('Gasto: {} - Gasto em aviões americanos: {} - Gasto em aviões britânicos: {}'.format(
        american_cost*american_planes+british_cost*british_planes,
        american_cost*american_planes,
        british_cost*british_planes)
         )
    print('Total de aviões: {} - Aviões americanos: {} - Aviões britânicos: {}'.format(american_planes+british_planes,
                                                                                       american_planes, 
                                                                                       british_planes)
         )
    print('Pessoal: {} - Pessoal americano: {} - Pessoal britânico: {}'.format(
        american_pers*american_planes+british_pers*british_planes,
        american_pers*american_planes,
        british_pers*british_planes)
         )  

In [6]:
#Resolvendo o problema para budget=300000
x,y = plane_model(300000)
plane_info(x,y,300000)

Informações
Cargo: 960000.0
Budget limite: 300000
Gasto: 288000.0 - Gasto em aviões americanos: 288000.0 - Gasto em aviões britânicos: 0.0
Total de aviões: 32 - Aviões americanos: 32 - Aviões britânicos: 0
Pessoal: 512 - Pessoal americano: 512 - Pessoal britânico: 0


In [7]:
#Uma solução por força bruta, para demonstrar que o método anterior é correto
#Consideraremos todos os pares ordenados de inteiros (x,y), em que x,y <= max_planes 
#(pois a primeira restrição impõe-lo imediatamente)

def plane_model_brute(budget, max_planes=44, max_pers=512,
                  american_cargo=30000.,american_pers=16, american_cost=9000.,
                  british_cargo=20000., british_pers=32, british_cost=5000.):
    feasible_solutions = []
    for x in range(max_planes+1):
        for y in range(max_planes+1):
            if x+y<= max_planes and american_pers*x+british_pers*y<=max_pers and american_cost*x+british_cost*y<=budget: 
                feasible_solutions.append((x,y))
    idx = np.argmax([cargo(X, american_cargo, british_cargo) for X in feasible_solutions])
    x,y = feasible_solutions[idx]   
    
    return int(x),int(y)

In [8]:
#Resolvendo o problema por força bruta para budget=300000
x,y = plane_model_brute(300000)
plane_info(x,y,300000)

Informações
Cargo: 960000.0
Budget limite: 300000
Gasto: 288000.0 - Gasto em aviões americanos: 288000.0 - Gasto em aviões britânicos: 0.0
Total de aviões: 32 - Aviões americanos: 32 - Aviões britânicos: 0
Pessoal: 512 - Pessoal americano: 512 - Pessoal britânico: 0
